In [ ]:
!pip install torcheval

In [ ]:
import numpy as np
import torch
import sklearn
import torcheval
from torcheval.metrics import R2Score

np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [ ]:
def f(x):
  return (1+x)**(1/2)

a) Computing random labels

In [ ]:
N=10000
xs=torch.rand(N)
labels=f(xs)

print(xs,"\n",labels)

b) Closed form solution

In [ ]:
X=torch.concat((torch.ones(N).unsqueeze(1),xs.unsqueeze(1)), dim=1)
print("X matrix", X)

w=(X.t()@X).inverse()@X.t()@labels
print("Weights", w)
print("labels comparison", X@w, labels)

c) sklearn linear regression

In [ ]:
lin_reg =sklearn.linear_model.LinearRegression()
lin_reg.fit(xs.unsqueeze(1), labels)
y_pred=lin_reg.predict(xs.unsqueeze(1))

print("Weights", lin_reg.coef_)
print("labels comparison", y_pred, labels)
print(sklearn.metrics.r2_score(labels,y_pred))

d) Grad descent from scratch

In [ ]:
xs.requires_grad=True
X.requires_grad=True
def gradient(x, y):
  loss=(y**2-x**2).mean()
  loss.backward()
  return loss.item()

In [ ]:
torch.manual_seed(42)

def gradient_descent():
    weights = torch.randn(2, requires_grad=True)
    lr = 0.1
    metric = R2Score()

    for i in range(200):
        y_pred = X @ weights
        loss = torch.mean((y_pred - labels) ** 2)

        loss.backward()

        with torch.no_grad():
            weights -= lr * weights.grad
            weights.grad.zero_()

        if i % 10 == 0:
            metric.reset()
            metric.update(y_pred.squeeze(), labels)
            print("iteration", i, "R^2:", metric.compute().item())
    return weights

weights = gradient_descent()


e) linear regression using pytorch

In [ ]:
class LinearRegression(torch.nn.Module):
  def __init__(self, input_dim, output_dim):
    super().__init__()
    self.linear = torch.nn.Linear(input_dim, output_dim)

  def forward(self, x):
    return self.linear(x)

In [ ]:
torch.manual_seed(42)
linreg = LinearRegression(1,1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(linreg.parameters(), lr=0.001)

metric = R2Score()

for epoch in range(1000):
  y_pred = linreg(xs.unsqueeze(1))
  loss = criterion(y_pred, labels.unsqueeze(1))

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  if epoch % 100 == 0:
    metric.reset()
    metric.update(y_pred, labels.unsqueeze(1))
    print(f"Epoch {epoch} R^2:", metric.compute().item())

f) Mini batch from scratch

In [ ]:
torch.manual_seed(42)
linreg2 = LinearRegression(1,1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(linreg2.parameters(), lr=0.001)

metric = R2Score()

indices = torch.randperm(N)
X_batches = torch.split(xs[indices], 32)
labels_batches = torch.split(labels[indices], 32)

for epoch in range(10):
  for x, y in zip(X_batches, labels_batches):
    x = x.detach()
    y = y.detach()

    y_pred = linreg2(x.unsqueeze(1))
    loss = criterion(y_pred, y.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  metric.reset()
  metric.update(y_pred, y.unsqueeze(1))
  print(f"Epoch {epoch} R^2:", metric.compute().item())

In [ ]:
print("closed form", w)
print("sklearn", weights)
print("scratch", linreg.linear.weight)
print("scratch mini batch", linreg2.linear.weight)

import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.set_figheight(10)
fig.set_figwidth(10)
xs_np=xs.detach().numpy()

ax.scatter(xs_np, labels.detach().numpy(), alpha=0.01, s=10, label="original data")

taylor = np.sqrt(1+0.5) + (1/(2*np.sqrt(1+0.5))) * (xs_np - 0.5)
ax.plot(xs_np, taylor, label="Taylor 1st order", linestyle="--", color="black")

ax.plot(xs_np, (X@w).detach().numpy(), label="closed form", linestyle="-", color="red")

ax.plot(xs_np, (X@weights).detach().numpy(), label="scratch", linestyle=":", color="blue")

pytorch_full = (xs.unsqueeze(1) @ linreg.linear.weight.t() + linreg.linear.bias).detach().numpy().squeeze()
ax.plot(xs_np, pytorch_full, label="pytorch full", linestyle="-.", color="green")

pytorch_mini = (xs.unsqueeze(1) @ linreg2.linear.weight.t() + linreg2.linear.bias).detach().numpy().squeeze()
ax.plot(xs_np, pytorch_mini, label="pytorch mini-batch", linestyle="--", color="purple")


leg = ax.legend(["original", "Taylor 1st order around 0.5", "closed form", "from scratch", "pytorch", "pytorch with mini batch from scratch"])

for lh in leg.legend_handles:
    lh.set_alpha(1)
    lh.set_linewidth(5)
plt.show()

In [ ]:
def mse(x,y):
  return ((x-y)**2).mean()

def taylor(x,a):
  return np.sqrt(1+a)+1/(2*np.sqrt(1+a))*(x-a)

# compare with taylor
print("original", mse(taylor(xs_np, 0.5) , labels.detach().numpy()))
print("closed form", mse(taylor(xs_np, 0.5) , (X@w).detach().numpy()))
print("from scratch", mse(taylor(xs_np, 0.5) , (X@weights).detach().numpy()))
print("pytorch", mse(taylor(xs_np, 0.5) , (xs.unsqueeze(1)@linreg.linear.weight.t()+linreg.linear.bias).detach().numpy()))
print("from scratch mini batch", mse(taylor(xs_np, 0.5) , (xs.unsqueeze(1)@linreg2.linear.weight.t()+linreg2.linear.bias).detach().numpy()))


# PART 2:
a) download data, keep 20% for testing

In [ ]:
# upload dataset
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

data=pd.read_csv("heart_disease_uci.csv")
y = (data.pop("num") > 0).astype(int)

# 2. Train/test split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    data, y, test_size=0.2, random_state=42, shuffle=True
)

# 3. preprocessing pipeline
numeric_features = X_train.select_dtypes(include=['int64','float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

# I use only imputer here to remove NaNs, in this case without normalization
numeric_transformer0 = sklearn.pipeline.Pipeline(steps=[
    ('imputer', sklearn.impute.SimpleImputer(strategy='mean')),
])

categorical_transformer0 = sklearn.pipeline.Pipeline(steps=[
    ('imputer', sklearn.impute.SimpleImputer(strategy='most_frequent')),
    ('onehot', sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

preprocessor0 = sklearn.compose.ColumnTransformer([
    ('num', numeric_transformer0, numeric_features),
    ('cat', categorical_transformer0, categorical_features)
])

# 4. Fit on training data, transform both
X_train0 = preprocessor0.fit_transform(X_train)
X_test0  = preprocessor0.transform(X_test)

b) Perform logistic regression and try to find the best hyperparameters (using sklearn).

In [ ]:
import time
# time now
start = time.time()

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [100, 1000, 10000]
}

logreg = sklearn.linear_model.LogisticRegression(random_state=42)

grid_search = sklearn.model_selection.GridSearchCV(logreg, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train0, y_train)

end = time.time()

print("convergence time", end - start)

print("Best parameters:", grid_search.best_params_)
print("Best cross-val accuracy:", grid_search.best_score_)
print("Test accuracy:", grid_search.score(X_test0, y_test))

c) Normalize data and run logistic regression again

In [ ]:
data=pd.read_csv("heart_disease_uci.csv")
y = (data.pop("num") > 0).astype(int)

# 2. Train/test split
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    data, y, test_size=0.2, random_state=42, shuffle=True
)

# 3. preprocessing pipeline
numeric_features = X_train.select_dtypes(include=['int64','float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

numeric_transformer = sklearn.preprocessing.StandardScaler()
categorical_transformer = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')

numeric_transformer = sklearn.pipeline.Pipeline(steps=[
    ('imputer', sklearn.impute.SimpleImputer(strategy='mean')),
    ('scaler', sklearn.preprocessing.StandardScaler())
])

categorical_transformer = sklearn.pipeline.Pipeline(steps=[
    ('imputer', sklearn.impute.SimpleImputer(strategy='most_frequent')),
    ('onehot', sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = sklearn.compose.ColumnTransformer([
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# 4. Fit on training data, transform both
X_train = preprocessor.fit_transform(X_train)
X_test  = preprocessor.transform(X_test)

In [ ]:
start = time.time()

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': [100, 1000, 10000]
}

logreg = sklearn.linear_model.LogisticRegression(random_state=42)

grid_search = sklearn.model_selection.GridSearchCV(logreg, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

end = time.time()
print("convergence time", end - start)

print("Best parameters:", grid_search.best_params_)
print("Best cross-val accuracy:", grid_search.best_score_)
print("Test accuracy:", grid_search.score(X_test, y_test))

From what we can see, though unnormalized data performed slightly better (0.853 against 0.847), it converged much slower (17 seconds against 0.5 seconds)

d) Try to find the best neural net to solve this problem which will have no more than
2 hidden layers (use sklearn). Without scaling

In [ ]:
hidden_layer_sizes = []

for i in range(3,8):
  for j in range(3,8):
    hidden_layer_sizes.append((2**i, 2**j))

param_grid = {
    'hidden_layer_sizes': hidden_layer_sizes
}

classifier = sklearn.neural_network.MLPClassifier(random_state=42, max_iter=100, early_stopping=True)

start = time.time()

grid_search = sklearn.model_selection.GridSearchCV(classifier, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train0, y_train)

end = time.time()

results_df = pd.DataFrame(grid_search.cv_results_)
sorted_df = results_df.sort_values(by="mean_test_score", ascending=False)

top5 = sorted_df[["mean_test_score", "params"]].head(5)
print(top5.to_string(index=False))

print("convergence time", end - start)
print("Best parameters:", grid_search.best_params_)
print("Best cross-val accuracy:", grid_search.best_score_)
print("Test accuracy:", grid_search.score(X_test0, y_test))

With normalization

In [ ]:
classifier = sklearn.neural_network.MLPClassifier(random_state=42, early_stopping=True)

start = time.time()

grid_search = sklearn.model_selection.GridSearchCV(classifier, param_grid, cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

end = time.time()

results_df = pd.DataFrame(grid_search.cv_results_)
sorted_df = results_df.sort_values(by="mean_test_score", ascending=False)

top5 = sorted_df[["mean_test_score", "params"]].head(5)
print(top5.to_string(index=False))

print("convergence time", end - start)
print("Best parameters:", grid_search.best_params_)
print("Best cross-val accuracy:", grid_search.best_score_)
print("Test accuracy:", grid_search.score(X_test, y_test))

It's interesting how normalization totally changes the architecture. We can notice that data normalization makes large difference in accuracy for both accuracy (0.728 against 0.858) and convergence time (3.85 and 5.36). Also notice how un-normalized data prefers "wider" network (64,32), though (32,128) again performs well in top 5. A logical assummption can be made that first network is more adapted to extracting raw data. On the contrary, the second one has better data exploration given the data is already "extracted".  

I have also tried changing learning-rate and activation function once, but because of long convergence time I changed the code. In short, smaller learning rates make small difference but not enough to trade-off with time. From sigmoid, tanh and relu, the best performer was always relu.

f) using NN classifier with PyTorch (no more than 2 hidden layers)

In [ ]:
# transforming numpy arrays to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

In [ ]:
class Classifier(torch.nn.Module):
  def __init__(self, in_ch, hidden1, hidden2, out_ch):
    super().__init__()

    self.nn = torch.nn.Sequential(
        torch.nn.Linear(in_ch, hidden1), torch.nn.ReLU(),
        torch.nn.Linear(hidden1, hidden2), torch.nn.ReLU(),
        torch.nn.Linear(hidden2, out_ch), torch.nn.ReLU(),
    )

  def forward(self, x):
    return self.nn(x)

In [ ]:
def train_classifier(hidden1, hidden2):
  model = Classifier(X_train.shape[1], hidden1, hidden2, y_train.shape[0])
  device = "cuda" if torch.cuda.is_available() else "cpu"
  model.to(device)

  loss_f = torch.nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

  num_epochs = 20

  train_accuracy_last = 0
  val_accuracy_last = 0

  best_validation_score = 0
  best_model = None
  patience = 50
  epochs_without_improvement=0

  for epoch in range(num_epochs):
      train_accuracy = []
      val_accuracy = []

      # train
      model.train()
      for batch in train_dataloader:
        x, y = batch
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_f(y_pred, y.long())

        train_accuracy.append(sklearn.metrics.f1_score(y.detach().cpu().numpy(), torch.argmax(y_pred, dim=1).detach().cpu().numpy(), average='weighted'))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

      train_accuracy_last = np.mean(train_accuracy)

      # test
      model.eval()

      with torch.no_grad():
        for batch in test_dataloader:
          x, y = batch
          x, y = x.to(device), y.to(device)
          y_pred = model(x)
          loss = loss_f(y_pred, y.long())

          val_accuracy.append(sklearn.metrics.f1_score(y.detach().cpu().numpy(), torch.argmax(y_pred, dim=1).detach().cpu().numpy(), average='weighted'))

        val_accuracy_last = np.mean(val_accuracy)

      # early stopping
      if val_accuracy_last > best_validation_score:
        best_validation_score = val_accuracy_last
        best_model = model.state_dict()
        epochs_without_improvement=0
      else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= patience:
          print(f"Early stopping triggered at epoch {epoch+1}")
          break

  if best_model is not None:
      model.load_state_dict(best_model)

  # print("hidden layers: ", hidden1, hidden2, "train accuracy", train_accuracy_last, "val accuracy", val_accuracy_last)
  return best_model, train_accuracy_last, val_accuracy_last

In [ ]:
best_validation = 0
best_model = None
best_hidden_layers=()

results ={} # train accuracy, validation accuracy, training time

for i in range(3,10):
  for j in range(3,10):
    train_start = time.time()
    model, train_accuracy, val_accuracy = train_classifier(2**i, 2**j)
    train_end = time.time()
    if val_accuracy > best_validation:
      best_validation = val_accuracy
      best_model = model
      best_hidden_layers=(2**i,2**j)
    results[(2**i,2**j)] = (train_accuracy, val_accuracy, train_end-train_start)

best_train = max(results.items(), key=lambda x: x[1][0])
print(f"Best training accuracy: {best_train[1][0]:.4f} at layers {best_train[0]}")

best_val = max(results.items(), key=lambda x: x[1][1])
print(f"Best validation accuracy: {best_val[1][1]:.4f} at layers {best_val[0]}")

fastest = min(results.items(), key=lambda x: x[1][2])
print(f"Fastest model: {fastest[1][2]:.2f}s at layers {fastest[0]}")

for layers, r in sorted(results.items(), key=lambda x: x[1][1], reverse=True):
  print(f"Layers: {layers}, Train accuracy: {r[0]:.4f}, Val accuracy: {r[1]:.4f}, Training time {r[2]:.4f}")

Best training accuracy: (512, 64), Train accuracy: 0.9795, Val accuracy: 0.8630, Training time 4.1472.  

Best validation accuracy: (8, 256), Train accuracy: 0.8906, Val accuracy: 0.8713, Training time 4.9603  

Fastest model: (64, 16), Train accuracy: 0.3860, Val accuracy: 0.4425, Training time 2.7956.

Sadly, as much as I tried to make an analysis of the results, most of the time they're quite different. For example, in one case I got the best validation accuracy being (64,8), which is the complete opposite of what we see now (8,256). This means that NN is highly dependent on the initial conditions. On the other hand, large first layer is almost always the best at training and thus at overfitting.  

Though (64,16) had faster training rate, it's one of the worst performers.

g) Compare accuracies of all obtained models.

In [ ]:
# after finding out that (8, 256) performs the best
# I runned several cases to find the best random initialization and avoid local minimums

best_model = None
best_val = 0.0
for _ in range(10):
  model, train_accuracy, validation_accuracy = train_classifier(8,256)
  print(train_accuracy, validation_accuracy)
  if validation_accuracy > best_val:
    best_model=model
    best_val = validation_accuracy

print(best_val)

Right now there are  

1) logistic regression without normalization 0.853, time 28.06s  
2) logistic regression with normalization 0.847, time 0.529s  
3) scikit learn nn without normalization 0.809, time 7.0s  
4) scikit learn nn with normalization 0.831, time 5.82s  
5) pytorch best model 0.858, time 4.9s  

Which is quite interesting that simple logistic regression with normalization performs almost as well as pytorch NN. We can also notice from this data that normalization increases convergence time in general.

h) Retrain your best-performing model using PyTorch Lightning. Incorporate a logger
to track your experiments and use a learning rate scheduler.

In [ ]:
import lightning as pl

In [ ]:
class LitClassifier(pl.LightningModule):
    def __init__(self, input_size, hidden1, hidden2, output_size, lr=1e-2):
        super().__init__()
        self.save_hyperparameters()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, hidden1),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden1, hidden2),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden2, output_size)
        )
        self.loss_f = torch.nn.CrossEntropyLoss()

    def forward(self, x):
        return self.model(x.float())

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.01)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss_f(y_pred, y.long())
        preds = torch.argmax(y_pred, dim=1)
        acc = sklearn.metrics.f1_score(y.cpu(), preds.cpu(), average='weighted')
        self.log("train_loss", loss, prog_bar=True)
        self.log("train_f1", acc, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x)
        loss = self.loss_f(y_pred, y.long())
        preds = torch.argmax(y_pred, dim=1)
        acc = sklearn.metrics.f1_score(y.cpu(), preds.cpu(), average='weighted')
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_f1", acc, prog_bar=True)


In [ ]:
pl_model = LitClassifier(X_train.shape[1], 8, 256, y_train.shape[0])

trainer = pl.Trainer(
    max_epochs=25,
    logger=pl.pytorch.loggers.tensorboard.TensorBoardLogger(save_dir="logs/"),
    callbacks=[pl.pytorch.callbacks.ModelCheckpoint(monitor="val_f1", mode="max")],
    deterministic=True
)

trainer.fit(pl_model, train_dataloader, test_dataloader)


Lightning model achieved 0.85 validation accuracy